In [1]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils.validation import check_array, check_is_fitted

import numpy as np
import matplotlib.pyplot as plt
import os
import copy
import inspect
import random
import shutil

from scipy import optimize
from sklearn import decomposition, cluster, model_selection, metrics
import sklearn

import deephyp
from deephyp import autoencoder
from deephyp import data

C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passi

In [2]:
def normalize(X):
    """ 
    Z normalization
    Make everything positive (NNLS does not work otherwise)
    """
    X = (X - X.mean(axis=0)) / X.std(axis=0)
    return X - X.min(axis=0)

In [3]:
kwargs = {'n_clusters': 3,
          'n_components': 20}

X = np.load("../data/HSI/Liver_map_150z25_60s_1TCPOBOP.npy", 'r')
Y = np.load("../data/HSI/Liver_map_150z25_60s_2TCPOBOP.npy", 'r')
Z = np.load("../data/HSI/Liver_map_150z25_60s_3OBOB.npy", 'r')

shape_X = X.shape 
shape_Y = Y.shape 
shape_Z = Z.shape 

X = copy.copy(X.reshape(-1, X.shape[-1]))
Y = copy.copy(Y.reshape(-1, Y.shape[-1]))
Z = copy.copy(Z.reshape(-1, Z.shape[-1]))

X = normalize(X)
Y = normalize(Y)
Z = normalize(Z)

(3750, 1000)

In [4]:
net = autoencoder.cnn_1D_network(inputSize=shape_X[2])

indices_train = random.sample(range(0,3750), k=3500)
indices_val = list(set(range(0,3750)) - set(indices_train))
dataTrain = data.Iterator(dataSamples=X[indices_train], targets=X[indices_train], batchSize=50)
dataVal = data.Iterator(dataSamples=X[indices_val], targets=X[indices_val], batchSize=50)

net = autoencoder.cnn_1D_network(inputSize=shape_X[2], activationFunc='relu')
net.add_train_op(name='csa', lossFunc='CSA', learning_rate=1e-3, decay_steps=None, decay_rate=None,
                 method='Adam', wd_lambda=0.0)

# create a directory to save the learnt model
model_dir = os.path.join('../data/HSI','autoencoder_cnn')
if os.path.exists(model_dir):
    # if directory already exists, delete it
    shutil.rmtree(model_dir)
os.mkdir(model_dir)

# train the network for 100 epochs, saving the model at epoch 50 and 100
net.train(dataTrain=dataTrain, dataVal=dataVal, train_op_name='csa', n_epochs=10, save_addr=model_dir,
          visualiseRateTrain=10, visualiseRateVal=10)






Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


epoch: 0, training loss: 1.92602
epoch: 0, validation loss: 1.51702
epoch: 10, training loss: 0.449796
epoch: 10, validation loss: 0.49487


ValueError: Parent directory of ../data/HSI\autoencoder_cnn\epoch_10\model.ckpt doesn't exist, can't save.

In [5]:
net
